# Chapter 2: Image classification

<br>

(2.1)=
## 1.1  Import dataset from Hugging Face

Usually data is not ready

Using the MNIST dataset, I created a CNN model that recognizes handwritten digits with an accuracy of 99.64%. That is, it correctly predicted 4,972 out of 5,000 images.







In [1]:
import torch
import numpy as np
from datasets import load_dataset

# set print options to avoid scientific notation
torch.set_printoptions(sci_mode=False)

# Load the MNIST dataset
ds = load_dataset("ylecun/mnist")
ds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

Based on the dataset structure, we can access the images like this:

In [2]:
first_image = ds['train'][0]['image']
first_image

Please note that the images in the dataset are in the PIL (Python Imaging Library) format.

In [3]:
type(first_image)

PIL.PngImagePlugin.PngImageFile

<br>

## 1.2 Data Preprocessing

Usually, the collected data is not immediately ready to be the input of a machine learning model. The steps taken to clean and prepare the data are known as **data preprocessing**.

When working with PIL images in deep learning, the typical workflow involves converting them first to NumPy arrays and then, to normalized PyTorch tensors. 

Suppose we have the following 4x4 PIL image:

```{figure} ../images/image-example.png
---
width: 180px
name: image-example
---
4x4 PIL image
```

When we convert this image to a NumPy array, each pixel value ranges from 0 (black) to 255 (white), with intermediate values representing varying shades of gray.

```python
np.array([[  0,  42,  85, 127],
          [ 42,  85, 127, 170],
          [ 85, 127, 170, 213],
          [127, 170, 213, 255]])
```

Neural networks perform better when when we **normalize** the input values so that they fall within the range [0, 1]. To do this, we divide each pixel value by 255, transforming the image into a format where 0 represents black and 1 represents white.

```python
tensor([[0.0000, 0.1647, 0.3333, 0.4980],
        [0.1647, 0.3333, 0.4980, 0.6667],
        [0.3333, 0.4980, 0.6667, 0.8353],
        [0.4980, 0.6667, 0.8353, 1.0000]])
```

To apply these transformations to all the images in the dataset, we will use a processing function along with the Hugging Face's `map()` function.

```{note}
- The `map()` function allows to apply a processing function to a dataset more efficiently than simply iteratively calling it for each example. For more information, please refer to the [Hugging Face documentation](https://huggingface.co/docs/datasets/process#map).
- The `num_proc` parameter sets the number of processes to use. **Multiprocessing** significantly speeds up processing by parallelizing tasks across multiple CPU cores.
```

In [22]:
# Process the first image manually to check the conversion
example = ds['train'][0]  # Get the first example in the dataset
processed_example = image_to_tensor(example)  # Apply the transformation function

print(type(processed_example["image"]))  # Should show <class 'torch.Tensor'>
print(processed_example["image"].shape)  # Should print torch.Size([28, 28])


<class 'torch.Tensor'>
torch.Size([28, 28])


In [38]:
def image_to_tensor(example):
    example["image"] = torch.tensor(np.array(example["image"])) / 255.0
    return example

train = []
for example in ds['train']:
    train.append(image_to_tensor(example))


In [41]:
def image_to_tensor(image):
    return torch.tensor(np.array(image)) / 255.0

In [45]:
train_x = []
train_y = []

for example in ds['train']:
    train_x.append(image_to_tensor(example['image']))
    train_y.append(example['label'])

In [46]:
len(train_x)

60000

In [47]:
train_x[2]

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000